In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [16]:
import pathlib

import pytest
import xarray as xr

import mesmer
from mesmer.mesmer_x import Expression, ConditionalDistributionOptions, ConditionalDistribution, ProbabilityIntegralTransform
from mesmer.core.datatree import map_over_datasets

In [3]:
scenario = "ssp585"
target_name = "tasmax"
option_2ndfit = False
save_files = False

In [4]:
# set some configuration parameters
THRESHOLD_LAND = 1 / 3
esm = "IPSL-CM6A-LR"

# TODO: replace with filefinder later
# load data
TEST_DATA_PATH = pathlib.Path("/Users/vbauer/Developer/mesmer/tests/test-data")
TEST_PATH = TEST_DATA_PATH / "output" / target_name / "one_scen_one_ens" / "test-params"
cmip6_data_path = mesmer.example_data.cmip6_ng_path()

In [5]:
# load predictor data
path_tas = cmip6_data_path / "tas" / "ann" / "g025"

fN_hist = path_tas / f"tas_ann_{esm}_historical_r1i1p1f1_g025.nc"
fN_ssp585 = path_tas / f"tas_ann_{esm}_{scenario}_r1i1p1f1_g025.nc"

time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
tas_hist = xr.open_dataset(fN_hist, decode_times=time_coder).drop_vars(
    ["height", "file_qf", "time_bnds"]
)
tas_ssp585 = xr.open_dataset(fN_ssp585, decode_times=time_coder).drop_vars(
    ["height", "file_qf", "time_bnds"]
)

tas = xr.DataTree.from_dict(
    {
        "historical": tas_hist,
        "ssp585": tas_ssp585,
    }
)

In [6]:
# make global mean
# global_mean_dt = map_over_subtree(mesmer.weighted.global_mean)
tas_glob_mean = mesmer.weighted.global_mean(tas)

# load target data
path_target = cmip6_data_path / target_name / "ann" / "g025"

fN_hist = path_target / f"{target_name}_ann_{esm}_historical_r1i1p1f1_g025.nc"
fN_ssp585 = path_target / f"{target_name}_ann_{esm}_{scenario}_r1i1p1f1_g025.nc"

time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
targ_hist = xr.open_dataset(fN_hist, decode_times=time_coder)
targ_ssp585 = xr.open_dataset(fN_ssp585, decode_times=time_coder)
# make sure times align
targ_hist["time"] = tas_hist["time"]
targ_ssp585["time"] = tas_ssp585["time"]

targ_data = xr.DataTree.from_dict(
    {
        "historical": targ_hist,
        "ssp585": targ_ssp585,
    }
)

In [7]:
def mask_and_stack(ds, threshold_land):
    ds = mesmer.mask.mask_ocean_fraction(ds, threshold_land)
    ds = mesmer.mask.mask_antarctica(ds)
    ds = mesmer.grid.stack_lat_lon(ds, stack_dim="gridpoint")
    return ds


# mask_and_stack_dt = map_over_subtree(mask_and_stack)
targ_data = mask_and_stack(targ_data, threshold_land=THRESHOLD_LAND)
pred_data = tas_glob_mean.copy()

weights = map_over_datasets(xr.ones_like, pred_data)
weights = map_over_datasets(lambda ds: ds.rename({"tas": "weights"}), weights)

In [8]:
# stacking
stacked_pred, stacked_targ, stacked_weights = (
    mesmer.core.datatree.broadcast_and_pool_scen_ens(
        predictors=pred_data,
        target=targ_data,
        weights=weights,
        member_dim=None,
    )
)

In [10]:
# declaring analytical form of the conditional distribution
expr = "norm(loc=c1 + c2 * __tas__, scale=c3)"
expr_name = "expr1"
expression_fit = Expression(
    expr, expr_name, boundaries_params={}, boundaries_coeffs={}
)

In [12]:
# preparing optimizers that will be used for first guess and training
myoptions = ConditionalDistributionOptions(
    options_optim=None,
    options_solver=None,
)

distrib = ConditionalDistribution(
    expression=expression_fit, options=myoptions
)

In [13]:
# find first guess
coeffs_fg = distrib.find_first_guess(
    predictors=stacked_pred,
    target=stacked_targ.tasmax,
    weights=stacked_weights.weights,
)
coeffs_fg

<xarray.Dataset> Size: 5kB
Dimensions:  (gridpoint: 118)
Coordinates:
    height   float64 8B 2.0
    lat      (gridpoint) float64 944B -49.5 -40.5 -31.5 -31.5 ... 76.5 76.5 76.5
    lon      (gridpoint) float64 944B 288.0 288.0 18.0 ... 306.0 324.0 342.0
Dimensions without coordinates: gridpoint
Data variables:
    c1       (gridpoint) float64 944B 6.488 -30.31 84.38 ... 133.5 58.93 -288.3
    c2       (gridpoint) float64 944B 0.9913 1.149 0.7714 ... 0.4913 0.744 1.975
    c3       (gridpoint) float64 944B 1.512 1.356 0.7257 ... 0.7664 1.195 1.585

In [14]:
# training the conditional distribution
# first round
distrib.fit(
    predictors=stacked_pred,
    target=stacked_targ.tasmax,
    weights=stacked_weights.weights,
    first_guess=coeffs_fg,
)
transform_coeffs = distrib.coefficients

In [15]:
# second round if necessary
if option_2ndfit:
    transform_coeffs = distrib.fit(
        predictors=stacked_pred,
        target=stacked_targ.tasmax,
        first_guess=transform_coeffs,
        weights=stacked_weights.weights,
        sample_dim="sample",
        smooth_coeffs=True,
        r_gasparicohn=500,
    )

transform_coeffs

<xarray.Dataset> Size: 5kB
Dimensions:  (gridpoint: 118)
Coordinates:
    height   float64 8B 2.0
    lat      (gridpoint) float64 944B -49.5 -40.5 -31.5 -31.5 ... 76.5 76.5 76.5
    lon      (gridpoint) float64 944B 288.0 288.0 18.0 ... 306.0 324.0 342.0
Dimensions without coordinates: gridpoint
Data variables:
    c1       (gridpoint) float64 944B 6.488 -30.31 84.38 ... 133.5 58.93 -288.3
    c2       (gridpoint) float64 944B 0.9913 1.149 0.7714 ... 0.4913 0.744 1.975
    c3       (gridpoint) float64 944B 1.512 1.356 0.7257 ... 0.7664 1.195 1.585
Attributes:
    expression_name:  expr1
    expression:       norm(loc=c1 + c2 * __tas__, scale=c3)

In [17]:
# probability integral transform on non-stacked data for AR(1) process
target_expression = Expression("norm(loc=0, scale=1)", "normal_dist")

pit = ProbabilityIntegralTransform(
    distrib_orig=distrib,
    distrib_targ=ConditionalDistribution(
        target_expression, ConditionalDistributionOptions()
    ),
)
transf_target = pit.transform(
    data=targ_data, target_name=target_name, preds_orig=pred_data, preds_targ=None
)

In [18]:
# training of auto-regression with spatially correlated innovations
local_ar_params = mesmer.stats.fit_auto_regression_scen_ens(
    transf_target,
    ens_dim="member",
    dim="time",
    lags=1,
)

# estimate covariance matrix
# prep distance matrix
geodist = mesmer.core.geospatial.geodist_exact(
    lon=targ_data["historical"].lon, lat=targ_data["historical"].lat
)
# prep localizer
LOCALISATION_RADII = range(1750, 2001, 250)
phi_gc_localizer = mesmer.stats.gaspari_cohn_correlation_matrices(
    geodist=geodist, localisation_radii=LOCALISATION_RADII
)


localized_ecov = mesmer.stats.find_localized_empirical_covariance(
    data=stacked_targ[target_name],
    weights=stacked_weights.weights,
    localizer=phi_gc_localizer,
    dim="sample",
    k_folds=30,
)

localized_ecov["localized_covariance_adjusted"] = mesmer.stats.adjust_covariance_ar1(
    localized_ecov.localized_covariance, local_ar_params.coeffs
)

/Users/vbauer/Developer/mesmer/mesmer/core/utils.py:99: OptimizeWarning: No local minimum found, returning the last element
  warnings.warn("No local minimum found, returning the last element", OptimizeWarning)


In [19]:
file_end = f"{target_name}_{expr_name}_{esm}_{scenario}"
distrib_file = TEST_PATH / "distrib" / f"params_transform_distrib_{file_end}.nc"
local_ar_file = TEST_PATH / "local_variability" / f"params_local_AR_{file_end}.nc"
localized_ecov_file = (
    TEST_PATH / "local_variability" / f"params_localized_ecov_{file_end}.nc"
)

if save_files:
    # save the parameters
    transform_coeffs.to_netcdf(distrib_file)
    local_ar_params.to_netcdf(local_ar_file)
    localized_ecov.to_netcdf(localized_ecov_file)
    pytest.skip("Updated param files.")